# Báo cáo đồ án môn Khoa học dữ liệu ứng dụng

Nhóm 6:
1. 18120212 - Huỳnh Long Nam
2. 18120438 - Nguyễn Đình Lộc

# Mô tả bài toán

## Giới thiệu

- Nhà ở có thể được xem là tài sản quan trọng nhất của đời người. Tuy nhiên, hiện nay, tình trạng giá nhà bị đẩy lên quá cao so với giá trị thực, khiến cho việc sở hữu một nơi an cư trở nên khó khăn với nhiều người. 

- Giá trị của một căn có thể được cấu thành từ nhiều yếu tố như: diện tích, đường đi, vị trí ( gần hay xa trung tâm), các tiện nghi có sẵn, mức độ hoàn thiện căn nhà,.. Bên cạnh đó, để định giá một căn nhà cần phải có các kiến thức chuyên môn về bất động sản, hiểu biết về thị trường nhà ở trong khu vực,.. việc mà không phải ai cũng làm được.

## Cuộc thi

- Cuộc thi: [House Prices - Advanced Regression Techniques](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques)

- Notebook mẫu: [#1 House Prices Solution [top 1%]](https://www.kaggle.com/code/jesucristo/1-house-prices-solution-top-1/notebook)

- Colab làm việc: [ADS-CQ18-Project Nhóm 6](https://colab.research.google.com/drive/1rXQsO-E1CZFtNFTnWXWZXf4ejHLim6gb)

- Nhóm lựa chọn cuộc thi **House Prices - Advanced Regression Techniques** cho đồ án của môn học với mong muốn sẽ hiểu rõ hơn về việc định giá một căn nhà, cũng học hỏi được một mô hình định giá nhà hiệu quả

- Cuộc thi huy tụ được hơn 4000 đội thi trên Kaggle và đã có nhiều đội giải quyết bài toán rất hiệu quả

- Notebook nhóm lựa chọn làm theo thuộc vào top 1%

## Cách tính điểm

Train:  1460 sales, and  81 features

Test:  1459 sales, and  80 features

Metric: Root-Mean-Squared-Error (RMSE)


# Giải quyết bài toán

Nhóm dựa trên notebook của tác giả, bắt đầu bằng việc tìm hiểu rõ về dataset: mô tả, EDA, kiểu dữ liệu, các giá trị bị thiếu,...

## 1.Khám phá dữ liệu

- Dữ liệu là thông tin về các căn hộ được rao bán tại thành phố Ames, bang Iowa, Mỹ trong khoảng thời gian từ năm 2006 đến năm 2010. 
- Dữ liệu gồm và 79 cột. Trong đó có 23 cột là dữ liệu phân loại, 23 cột dữ liệu phân cấp và 33 cột dữ liệu số.
- Tập train có 1460 dòng và tập test có 1459 dòng.
- Các cột dữ liệu số đáng chú ý gồm có:
  1.   GrLiveArea: diện tích sinh hoạt không bao gồm tầng hầm.
  2.   GarageArea: diện tích ga-ra.
  3.   TotalBsmtSF: tổng diện tích tầng hầm.
  4.   1stFlrSF: diện tích tầng 1.
  5.   TotRmsAbvGrd: tổng số phòng (không tính phòng tắm) không nằm dưới hầm.
  6.   MasVnrArea: diện tích chân tường.
- Các cột dữ liệu phân cấp đáng chú ý gồm có:
  1.   OverallQual: chất lượng hoàn thiệt và vật liệu tổng thể của căn nhà.
  2.   ExterQual: chất lượng ngoại thất.
  3.   BsmtQual: chiều cao của tầng hầm.
  4.   KitchenQual: chất lượng của nhà bếp.
  5.   GarageFinish: chất lượng hoàn thiện bên trong ga-ra.
  6.   FireplaceQu: chất lượng lò sưởi.
  7.   HeatingQC: chất lượng và tình trạng của hệ thống sưởi (nước và không khí)- Các cột dữ liệu phân loại đáng chú ý:
  1.   Neighborhood: những địa điểm ở thành phố Ames.
  2.   Exterior1st: cách bao phủ mặt bên ngoài của tường.
- Các cột phía trên chứa dữ liệu có mức độ tương quan lớn với giá thành của căn nhà.
- Ngoài ra còn có các cột dữ liệu bị thiếu có hàm ý:
  *   Alley: kiểu hẻm mặt tiền căn nhà. => Căn nhà không nằm trong hẻm thì sẽ để trống.
  *   BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2: các thuộc tính liên quan tới tầng hầm của căn nhà => Không có tầng hầm sẽ để trống.
  *   FireplaceQu: chất lượng lò sưởi => Không có lò sưởi sẽ để trống.
  *   GarageType, GarageFinish, GarageQual, GarageCond: thuộc tính liên quan tới ga-ra => Không có ga-ra sẽ để trống.
  *   PoolQC: chất lượng hồ bơi => Không có hồ bơi sẽ để trống.
  *   Fence: chất lượng hàng rào => Không có hàng rào sẽ để trống.
  *   MiscFeature: những đặc trưng khác của căn nhà mà không nằm trong các thuộc tính trong bảng => Không có sẽ để trống.









# 3. Tiền xử lý

## Note
Notebook: https://www.kaggle.com/code/jesucristo/1-house-prices-solution-top-1/notebook

